<a href="https://colab.research.google.com/github/geeky-programer/llm-agents/blob/main/llm_agent_nvd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [ ]:
!pip install beautifulsoup4

In [ ]:
import re
import requests
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
import json
from bs4 import BeautifulSoup

# CVE Parsing Function
def parse_cve_from_query(user_query):
    cve_pattern = r'CVE-\d{4}-\d{4,}'
    cves = re.findall(cve_pattern, user_query, re.IGNORECASE)
    return cves

# Function to Fetch CVE Details from NVD API
def fetch_cve_details(cve_id, api_key):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0"
    headers = {"apiKey": api_key}  # Replace `api_key` with your NVD API key
    params = {"cveid": cve_id}
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code, "message": response.text}

# Function to Summarize CVE Details
def summarize_cve(data):
    cve = data["vulnerabilities"][0]["cve"]

    # Basic Info
    cve_id = cve["id"]
    description = cve["descriptions"][0]["value"]
    severity = cve["metrics"]["cvssMetricV31"][0]["cvssData"]["baseSeverity"]
    score = cve["metrics"]["cvssMetricV31"][0]["cvssData"]["baseScore"]
    attack_vector = cve["metrics"]["cvssMetricV31"][0]["cvssData"]["attackVector"]
    published_date = cve["published"]
    modified_date = cve["lastModified"]

    # Affected Products
    affected_products = [
        node["criteria"]
        for config in cve["configurations"]
        for node in config["nodes"]
        for node in node.get("cpeMatch", [])
    ]

    # References
    references = [ref["url"] for ref in cve.get("references", [])]

    summary = f"""
    CVE ID: {cve_id}
    Description: {description}
    Severity: {severity} (Score: {score})
    Attack Vector: {attack_vector}
    Published: {published_date}
    Last Modified: {modified_date}
    Affected Products: {', '.join(affected_products)}
    References: {', '.join(references)}
    """
    return summary.strip()


# LangChain Tool for Parsing CVEs
def parse_tool(query: str):
    cves = parse_cve_from_query(query)
    if not cves:
        return "No CVE identifiers found in the query."
    return f"Found CVE IDs: {', '.join(cves)}"

# LangChain Tool for Fetching and Summarizing CVE Details
def fetch_tool(cve_id: str):
    api_key = ""  # Replace with your actual API key
    cve_data = fetch_cve_details(cve_id, api_key)
    return summarize_cve(cve_data)


def fetch_reference_info(url, tags):

    if "Permissions Required" in tags:
        return None  # Skip this reference

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            title = soup.title.string if soup.title else "No title found"
            paragraphs = soup.find_all("p")
            summary = paragraphs[0].text.strip() if paragraphs else "No content found"
            return f"Title: {title}\nSummary: {summary}\nURL: {url}"
        else:
            return f"Error fetching {url}: HTTP {response.status_code}"
    except Exception as e:
        return f"Error fetching {url}: {e}"

def get_cve_details_with_references(cve_id):

    # Fetch CVE details
    api_key = ""
    cve_data = fetch_cve_details(cve_id,api_key)
    summary = summarize_cve(cve_data)

    # Fetch reference URLs and extract info
    vulnerabilities = cve_data.get("vulnerabilities", [])
    if not vulnerabilities:
        return summary

    references = vulnerabilities[0]["cve"].get("references", [])
    if not references:
        return f"{summary}\n\nNo references available for this CVE."

    # Filter out references with "Permissions Required" tag
    reference_details = []
    for ref in references:
        url = ref["url"]
        tags = ref.get("tags", [])
        ref_info = fetch_reference_info(url, tags)
        if ref_info:  # Only add reference if not None (i.e., it doesn't have "Permissions Required")
            reference_details.append(ref_info)

    if reference_details:
        reference_section = "\n\n".join(reference_details)
        return f"{summary}\n\nAdditional Information from References:\n{reference_section}"
    else:
        return summary



# Define Tools for the Agent
tools = [
    Tool(name="Parse CVE IDs", func=parse_tool, description="Extract CVE IDs from the user query."),
    Tool(name="Fetch CVE Details", func=fetch_tool, description="Fetch CVE details by providing a CVE ID."),
    Tool(name="Fetch CVE References", func=get_cve_details_with_references, description="Fetch reference URLs and extra details for a given CVE ID.")
]



In [ ]:
# Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-4",openai_api_key='')

# Initialize Agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Example User Query
user_query = "Can you check details for CVE-2023-50423 with references?"
response = agent.run(user_query)
print(response)




> Entering new AgentExecutor chain...
The user is asking for details and references for a specific CVE ID. I need to first validate the CVE ID and then fetch the details and references for it.
Action: Parse CVE IDs
Action Input: CVE-2023-50423
Observation: Found CVE IDs: CVE-2023-50423
Thought:The CVE ID is valid. Now I need to fetch the details for this CVE.
Action: Fetch CVE Details
Action Input: CVE-2023-50423
Observation: CVE ID: CVE-2023-50423
    Description: SAP BTP Security Services Integration Library ([Python] sap-xssec) - versions < 4.1.0, allow under certain conditions an escalation of privileges. On successful exploitation, an unauthenticated attacker can obtain arbitrary permissions within the application.
    Severity: CRITICAL (Score: 9.8)
    Attack Vector: NETWORK
    Published: 2023-12-12T02:15:08.797
    Last Modified: 2024-09-28T23:15:12.360
    Affected Products: cpe:2.3:a:sap:sap-xssec:*:*:*:*:*:python:*:*
    References: https://blogs.sap.com/2023/12/12/unveil